In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [2]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
path = Path('../Resources/finished_data_dict.pkl')
data_dict_import = load_obj(path)
path = Path('../Resources/finished_key_list.csv')
key_list_import_df = pd.read_csv(path,index_col=0)
key_list = []
for i in key_list_import_df['Symbol']:
    key_list.append(i)
len(key_list)

9992

In [5]:
ftd_key_list = []
etf_key_list = []
eqt_key_list = []
float_count_is_0 = 0

for i in key_list:
    index_value = i
    float_ftd_pct_ytd = data_dict_import[index_value]['ftd_stats']['float_ftd_pct_ytd']
    ## Check if there's float data. Otherwise, calculate using Outstanding share data
    ## ETFs have no float data, so they always use Outstanding Shares 
    if float_ftd_pct_ytd == 0:        
        os_ftd_pct_ytd = data_dict_import[index_value]['ftd_stats']['os_ftd_pct_ytd']
        if os_ftd_pct_ytd >= 5:
            float_count_is_0 += 1
            ftd_key_list.append(i)
            isEtf = data_dict_import[index_value]['companyProfile']['isEtf']
            if isEtf == True:
                etf_key_list.append(i)
            else:
                eqt_key_list.append(i)
    else:
        if float_ftd_pct_ytd >= 5:
            ftd_key_list.append(i)
            isEtf = data_dict_import[index_value]['companyProfile']['isEtf']
            if isEtf == 'True':
                etf_key_list.append(i)
            else:
                eqt_key_list.append(i)

print(f'Float = 0 for '+str(float_count_is_0)+' symbols')
print('All list length:',len(ftd_key_list))
print('ETF list length:',len(etf_key_list))
print('EQT list length:',len(eqt_key_list))

Float = 0 for 2112 symbols
All list length: 3695
ETF list length: 1872
EQT list length: 1823


In [6]:
test_symbol = 'GME'
symbol = test_symbol

In [7]:
df = data_dict_import[symbol]['dataFrame'].copy()
X = df.drop(columns={'close','adjClose'}).values
y = df['close'].values

# # Create training and testing datasets
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

scaler = StandardScaler().fit(X)
X = scaler.transform(X) 

# Create a sequential model
nn = Sequential()

num_of_inputs = 16
num_of_outputs= 1

# First hidden layer
nn.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

# Second hidden layer
nn.add(Dense(units=8, activation="relu"))

# Output layer
nn.add(Dense(units=num_of_outputs, activation="linear"))

# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model = nn.fit(X, y, validation_split=0.3, epochs=800, verbose=0)

# Save model_2 as JSON
nn_json = nn.to_json()

file_path = Path('../Model_Data/'+symbol+'_model_2Layers.json')
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

# Save weights
file_path = Path('../Model_Data/'+symbol+'_model_2Layers_weights.h5')
nn.save_weights(file_path)

In [14]:
# load json and create model
file_path = Path('../Model_Data/'+symbol+'_model_2Layers.json')
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = Path('../Model_Data/'+symbol+'_model_2Layers_weights.h5')
loaded_model.load_weights(file_path)

df2 = df.copy()
df2["predicted"] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]
df_pred

,close,predicted
Date,,
2016-01-04,28.309999,28.336760
2016-01-05,28.770000,28.667849
2016-01-06,28.370001,28.201527
2016-01-07,28.450001,28.441654
2016-01-08,28.370001,28.320555
...,...,...
2021-10-25,173.970000,172.639984
2021-10-26,177.840000,178.294830
2021-10-27,173.510000,170.671295


In [9]:
model_loss, model_accuracy = nn.evaluate(X, y, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

46/46 - 0s - loss: 6.1345 - mse: 6.1345 - 40ms/epoch - 879us/step
Loss: 6.1345367431640625, Accuracy: 6.1345367431640625


In [10]:
model_accuracy

6.1345367431640625

In [12]:
len(key_list)

9992

In [13]:
test_key_list = key_list[0:10]
len(test_key_list)

10

In [11]:
for symbol in test_key_list:
    df = data_dict_import[symbol]['dataFrame'].copy()
    X = df.drop(columns={'close','adjClose'}).values
    y = df['close'].values
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X) 
    num_of_inputs = 16
    num_of_outputs= 1
    ## Run Model 5 times and save best model 
    for i in range(5):
        ## Initialize Model 
        nn = Sequential()
        
    # First hidden layer
    nn.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

    # Second hidden layer
    nn.add(Dense(units=8, activation="relu"))

    # Output layer
    nn.add(Dense(units=num_of_outputs, activation="linear"))

    # Compile the model
    nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

    # Fit the model
    model = nn.fit(X, y, validation_split=0.3, epochs=800, verbose=0)

#     # Save model as JSON
#     nn_json = nn.to_json()

#     file_path = Path('../Model_Data_Temp/Temp_model_2Layers.json')
#     with open(file_path, "w") as json_file:
#         json_file.write(nn_json)

#     # Save weights
#     file_path = Path('../Model_Data_Temp/Temp_model_2Layers_weights.h5')
#     nn.save_weights(file_path)
    
#     # load json and create model
#     file_path = Path('../Model_Data/Temp_model_2Layers.json')
#     with open(file_path, "r") as json_file:
#         model_json = json_file.read()
#     loaded_model = model_from_json(model_json)

#     # load weights into new model
#     file_path = Path('../Model_Data/Temp_model_2Layers_weights.h5')
#     loaded_model.load_weights(file_path)

#     df2 = df.copy()
#     df2["predicted"] = loaded_model.predict(X)
#     df_pred = df2[['close','predicted']]
    
    model_loss, model_accuracy = nn.evaluate(X, y, verbose=2)
    
    if i = 0:
        model_accuracy1 = model_accuracy
        # Save model as JSON
        nn_json = nn.to_json()

        file_path = Path('../Model_Data/'+symbol+'_model_2Layers.json')
        with open(file_path, "w") as json_file:
            json_file.write(nn_json)

        # Save weights
        file_path = Path('../Model_Data/'+symbol+'_model_2Layers_weights.h5')
        nn.save_weights(file_path)
    else:
        if model_accuracy > model_accuracy1:
            model_accuracy1 = model_accuracy
            # Save model as JSON
            nn_json = nn.to_json()

            file_path = Path('../Model_Data/'+symbol+'_model_2Layers.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = Path('../Model_Data/'+symbol+'_model_2Layers_weights.h5')
            nn.save_weights(file_path)
    

0
1
2
3
4
